# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,Lebu,CL,-37.6167,-73.6500,10.09,67,100,6.42,1722099511
1,1,Taoudenni,ML,22.6783,-3.9836,45.73,7,3,2.13,1722099696
2,2,Albany,US,42.6001,-73.9662,28.30,51,0,0.45,1722099696
3,3,Port-aux-Français,TF,-49.3500,70.2167,0.68,67,99,14.93,1722099696
4,4,Chicoana,AR,-25.1009,-65.5331,22.25,43,0,2.61,1722099696


In [3]:
city_data_df.describe

<bound method NDFrame.describe of      City_ID               City Country      Lat       Lng  Max Temp  \
0          0               Lebu      CL -37.6167  -73.6500     10.09   
1          1          Taoudenni      ML  22.6783   -3.9836     45.73   
2          2             Albany      US  42.6001  -73.9662     28.30   
3          3  Port-aux-Français      TF -49.3500   70.2167      0.68   
4          4           Chicoana      AR -25.1009  -65.5331     22.25   
..       ...                ...     ...      ...       ...       ...   
578      578           Tangalle      LK   6.0240   80.7911     26.93   
579      579           Itaituba      BR  -4.2761  -55.9836     38.27   
580      580            Urangan      AU -25.3000  152.9000     18.39   
581      581           Tshikapa      CD  -6.4167   20.8000     29.20   
582      582          Boyolangu      ID  -8.1181  111.8935     22.77   

     Humidity  Cloudiness  Wind Speed        Date  
0          67         100        6.42  1722099511

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     583 non-null    int64  
 1   City        583 non-null    object 
 2   Country     579 non-null    object 
 3   Lat         583 non-null    float64
 4   Lng         583 non-null    float64
 5   Max Temp    583 non-null    float64
 6   Humidity    583 non-null    int64  
 7   Cloudiness  583 non-null    int64  
 8   Wind Speed  583 non-null    float64
 9   Date        583 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 45.7+ KB


In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    scale=1, 
    color="City", 
    alpha=0.5, 
    tiles="OSM", 
    width=700, 
    height=500, 
    title="City Humidity Map"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
4,4,Chicoana,AR,-25.1009,-65.5331,22.25,43,0,2.61,1722099696
7,7,Fallon,US,39.4735,-118.7774,25.62,21,0,2.68,1722099697
32,32,Belyy Yar,RU,53.6039,91.3903,22.94,73,0,3.00,1722099699
119,119,Jamestown,US,42.0970,-79.2353,26.40,46,0,2.06,1722099579
202,202,Gallup,US,35.5281,-108.7426,24.91,47,0,3.09,1722099882


In [7]:
ideal_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17 entries, 4 to 570
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     17 non-null     int64  
 1   City        17 non-null     object 
 2   Country     17 non-null     object 
 3   Lat         17 non-null     float64
 4   Lng         17 non-null     float64
 5   Max Temp    17 non-null     float64
 6   Humidity    17 non-null     int64  
 7   Cloudiness  17 non-null     int64  
 8   Wind Speed  17 non-null     float64
 9   Date        17 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 1.5+ KB


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
4,Chicoana,AR,-25.1009,-65.5331,43,
7,Fallon,US,39.4735,-118.7774,21,
32,Belyy Yar,RU,53.6039,91.3903,73,
119,Jamestown,US,42.0970,-79.2353,46,
202,Gallup,US,35.5281,-108.7426,47,
298,Luau,AO,-10.7073,22.2247,12,
360,Sydney Mines,CA,46.2367,-60.2177,68,
418,Helena Valley Southeast,US,46.6153,-111.9216,26,
424,Grand Falls-Windsor,CA,48.9332,-55.6649,45,
463,Masuguru,TZ,-11.3667,38.4167,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row["Lat"], row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Chicoana - nearest hotel: No hotel found
Fallon - nearest hotel: No hotel found
Belyy Yar - nearest hotel: No hotel found
Jamestown - nearest hotel: No hotel found
Gallup - nearest hotel: No hotel found
Luau - nearest hotel: No hotel found
Sydney Mines - nearest hotel: No hotel found
Helena Valley Southeast - nearest hotel: No hotel found
Grand Falls-Windsor - nearest hotel: No hotel found
Masuguru - nearest hotel: No hotel found
Ukiah - nearest hotel: No hotel found
Ada - nearest hotel: No hotel found
Marsannay-la-Côte - nearest hotel: No hotel found
Gornji Vakuf - nearest hotel: No hotel found
Concordia - nearest hotel: No hotel found
Newman - nearest hotel: No hotel found
Meridian - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,Chicoana,AR,-25.1009,-65.5331,43,No hotel found
7,Fallon,US,39.4735,-118.7774,21,No hotel found
32,Belyy Yar,RU,53.6039,91.3903,73,No hotel found
119,Jamestown,US,42.0970,-79.2353,46,No hotel found
202,Gallup,US,35.5281,-108.7426,47,No hotel found
298,Luau,AO,-10.7073,22.2247,12,No hotel found
360,Sydney Mines,CA,46.2367,-60.2177,68,No hotel found
418,Helena Valley Southeast,US,46.6153,-111.9216,26,No hotel found
424,Grand Falls-Windsor,CA,48.9332,-55.6649,45,No hotel found
463,Masuguru,TZ,-11.3667,38.4167,25,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    width=700,
    height=500,
    title="Hotel Locations",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)